In [12]:
import ipywidgets as widgets 
from IPython.display import display
from threading import Thread
from queue import Queue

messages = Queue()
recordings = Queue()

record_button = widgets.Button(
    description = "Record",
    disabled = False,
    button_style = "success",
    icon="microphone"
)

stop_button = widgets.Button(
    description = "Stop",
    disabled = False,
    button_style = "warning",
    icon="stop"
)

output = widgets.Output()

def start_recording(data):
    messages.put(True)
    
    with output:
        display("Starting...")
        record = Thread(target=record_microphone)
        record.start()
        
        transcribe = Thread(target=speech_recognition, args=(output,))
        transcribe.start()

def stop_recording(data):
    with output:
        messages.get()
        display("Stopped.")

record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display(record_button, stop_button, output)

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle())

Output()

In [13]:
!pip install pyaudio

     -------------------------------------- 164.1/164.1 kB 1.4 MB/s eta 0:00:00


In [14]:
import pyaudio

p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))
    
p.terminate()
    

{'index': 0, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Input', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 1, 'structVersion': 2, 'name': 'Microphone Array (Realtek(R) Au', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 2, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Output', 'hostApi': 0, 'maxInputChannels': 0, 'maxOutputChannels': 2, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 3, 'structVersion': 2, 'name': 'Speakers (Realtek(R) Audio)', 'hostApi': 0, 'maxInputChannels'

In [ ]:
CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS = 20 
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

def record_microphone(chunk=1024):
    p = pyaudio.PyAudio()
    
    stream = p.open(format= AUDIO_FORMAT,
                   channels=CHANNELS,
                   rate=FRAME_RATE,
                   input=True,
                   input_device_index=2,
                   frames_per_buffer= chunk)
    frames = []
    
    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)
        
        if len(frames) >= (FRAME_RATES + RECORD_SECONDS) / chunk:
            recordings.put(frames.copy())
            frames = []
            
            
    stream.stop_stream()
    stream.close()
    p.terminate()
    

In [15]:
!pip install vosk

     ---------------------------------------- 14.0/14.0 MB 3.1 MB/s eta 0:00:00
     -------------------------------------- 163.3/163.3 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22521 sha256=b7b7510aa34e851b577418a8289993b79327d5bcfad794538fa3b51e89d9a92b
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\69\cb\6a\5e5977c5c1fab3b94ff429718103884855b0d5671ed9880100
Successfully built srt


In [16]:
!pip install transformers

     ---------------------------------------- 9.7/9.7 MB 4.9 MB/s eta 0:00:00
     -------------------------------------- 450.7/450.7 kB 2.6 MB/s eta 0:00:00
     -------------------------------------- 303.8/303.8 kB 2.7 MB/s eta 0:00:00
     ---------------------------------------- 2.4/2.4 MB 2.8 MB/s eta 0:00:00
     -------------------------------------- 183.9/183.9 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.7.1
    Uninstalling fsspec-2022.7.1:
      Successfully uninstalled fsspec-2022.7.1


In [17]:
!pip install torch

     -------------------------------------- 203.0/203.0 MB 1.9 MB/s eta 0:00:00
  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
     ---------------------------------------- 6.2/6.2 MB 3.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0
  Attempting uninstall: sympy
    Found existing installation: sympy 1.10.1
    Uninstalling sympy-1.10.1:
      Successfully uninstalled sympy-1.10.1


In [ ]:
import subprocess
import json 
from vosk import Model, KaldiRecognizer 

model = Model(model_name="vosk-model-enus-0.22")
rec = KaldiRecognizer(model, FRAME_RATES)
rec.SetWords(True)

def speech_recognition(output):
    while not messages.empty():
        frames = recording.get()
        
        rec.AcceptWaveform(b''.join(frames))
        result = rec.Result()
        text = json.loads(result)["text"]
        
        cased = subprocess.check_ouput("python recasepunc.py predict recasepunc/checkpoint", shell=True, text=True, input=text)
        output.append_stdout(cased)
        